In [46]:
import cv2
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from skimage.feature import hog
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage import data, exposure
from sklearn import datasets, svm, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances

In [33]:
#
# Color histogram as Global Feature
#
def compute_color_histogram(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    hist_h = cv2.calcHist([hsv], [0], None, [256], [0, 256])
    hist_s = cv2.calcHist([hsv], [1], None, [256], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], None, [256], [0, 256])
    hist = np.vstack((hist_h, hist_s, hist_v)).flatten()
    return hist

    # plt.figure(figsize=(10,5))
    # plt.title("Color Histogram (HSV)")
    # plt.xlabel("Bins")
    # plt.ylabel("# of Pixels")
    # plt.plot(hist_h, color='r', label='Hue')
    # plt.plot(hist_s, color='g', label='Saturation')
    # plt.plot(hist_v, color='b', label='Value')
    # plt.legend()
    # plt.xlim([0, 256])
    # plt.show()

#
# SIFT keypoints as Local Feature
#
def compute_sift_features(image, nfeatures=200):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create(nfeatures=nfeatures)
    # sift.setContrastThreshold(0.04)
    # sift.setEdgeThreshold(10)
    # sift.setSigma(1.6)
    # print(sift.getContrastThreshold())
    # print(sift.getEdgeThreshold())
    # print(sift.getSigma())
    
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    # print(f"Number of keypoints: {len(keypoints)}")
    # print(descriptors.shape)
    
    keypoints = keypoints[:nfeatures]
    descriptors = descriptors[:nfeatures]
    sift_image = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    # Display the image with keypoints
    # plt.figure(figsize=(10,10))
    # plt.title(f"SIFT Keypoints: {len(keypoints)} detected")
    # plt.imshow(cv2.cvtColor(sift_image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()
    return descriptors

def main():
    image_path = 'C:\\source\\repos\\cv-assignment\\5-Categories\\buddha\\image_0001.jpg'
    image = cv2.imread(image_path)

    # plt.figure(figsize=(10,10))
    # plt.title("Original Image")
    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()
    
    compute_color_histogram(image)
    # compute_sift_features(image)

if __name__ == "__main__":
    main()

In [34]:
def ComputeXY(train_folder='', allowed_sports=[], HyperParams={}):
    X = np.array([])
    y = np.array([])
    for folder_name in os.listdir(train_folder):
        if folder_name not in allowed_sports: continue
        folder_dir = os.path.join(train_folder, folder_name)
        print(f"Reading images from folder: {folder_name}")
        for image_name in os.listdir(folder_dir):
            image_path = os.path.join(folder_dir, image_name)
            if os.path.isfile(image_path):
                image = cv2.imread(image_path)
                #
                histo_X = compute_color_histogram(image)
                sift_X = compute_sift_features(image, nfeatures=HyperParams['SIFT_NFEATURES']).flatten() # returns a 2D array of descriptors
                max_len = 128 * HyperParams['SIFT_NFEATURES'] # TODO MAGIC NUM 128
                if len(sift_X) < max_len:
                    padding = np.zeros(max_len - len(sift_X))
                    sift_X = np.concatenate((sift_X, padding))
                X_one = np.concatenate((sift_X, histo_X))
                #
                X = X_one if X.size == 0 else np.vstack((X, X_one))
                #
                one_hot_encoding = np.zeros(len(allowed_sports))
                one_hot_encoding[allowed_sports.index(folder_name)] = 1
                y = one_hot_encoding if y.size == 0 else np.vstack((y, one_hot_encoding))
    return X, y
    

In [48]:
TRAIN_FOLDER = 'C:\\Users\\sseksaria\\Downloads\\SportsData\\train'
ALLOWED_SPORTS = ['air hockey', 'ampute football', 'archery', 'arm wrestling']#, 'axe throwing', 'balance beam', 'barell racing', 'baseball', 'basketball', 'baton twirling']
HyperParams = {
    'SIFT_NFEATURES': 200
}

X, y = ComputeXY(TRAIN_FOLDER, ALLOWED_SPORTS, HyperParams)
print(f'X: {X.shape}')
print(f'y: {y.shape}')



Reading images from folder: air hockey
Reading images from folder: ampute football
Reading images from folder: archery
Reading images from folder: arm wrestling
X: (455, 26368)
y: (455, 4)


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
multi_classifier = MultiOutputClassifier(SVC())
multi_classifier.fit(X_train, y_train)

y_pred = multi_classifier.predict(X_test)
confusion_matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Confusion Matrix:")
print(confusion_matrix)
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Accuracy:", accuracy)
classification_rep = metrics.classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Classification Report:")
print(classification_rep)

Confusion Matrix:
[[25  1  0  1]
 [ 7 10  0  1]
 [18  3  3  3]
 [ 9  0  0 10]]
Accuracy: 0.5274725274725275
Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.93      0.58        27
           1       0.71      0.56      0.62        18
           2       1.00      0.11      0.20        27
           3       0.67      0.53      0.59        19

    accuracy                           0.53        91
   macro avg       0.70      0.53      0.50        91
weighted avg       0.70      0.53      0.48        91

